In [ ]:
import os
import numpy as np
import json
import keras
from keras import backend as k
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

data_path = '/home/rbodo/.snntoolbox/Datasets/imagenet'
train_path = os.path.join(data_path, 'training')
test_path = os.path.join(data_path, 'validation')
class_idx_path = os.path.join(data_path, 'imagenet_class_index_dir.json')
save_path = '/home/rbodo/.snntoolbox/data/imagenet/inception'

Using Theano backend.


Using cuDNN version 5005 on context None


Mapped name None to device cuda: GeForce GTX 980 Ti (0000:01:00.0)


In [ ]:
path_to_orig_model = os.path.join(save_path, '76.28_93.03.h5')
if os.path.isfile(path_to_orig_model):
    model = keras.models.load_model(path_to_orig_model)
else:
    # create the base pre-trained model
    model = InceptionV3()
    
    # Flip kernels (tensorflow uses correlation instead of convolution, like caffe)
    to_assign = []
    for layer in model.layers:
        if 'conv2d' in layer.name:
            original_kernel = k.get_value(layer.kernel)
            converted_kernel = original_kernel[::-1, ::-1, :, :]
            to_assign.append((layer.kernel, converted_kernel))
    k.batch_set_value(to_assign)
    
    model.save(path_to_orig_model)

In [ ]:
target_size = (299, 299)
batch_size = 20

class_idx = json.load(open(class_idx_path, "r"))
classes = [class_idx[str(idx)][0] for idx in range(len(class_idx))]

datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_dataflow = datagen.flow_from_directory(
    train_path, target_size=target_size, classes=classes, batch_size=batch_size)

test_dataflow = datagen.flow_from_directory(
    test_path, target_size=target_size, classes=classes, batch_size=batch_size)

In [ ]:
from keras.constraints import unit_norm, max_norm
from keras.regularizers import l2

for layer in model.layers:
    # if 'conv' in layer.name:
    #     print("Modifying Conv layer")
    #     layer.kernel_constraint = unit_norm()
    #     layer.kernel_regularizer = l2(0.01)
    if 'batch' in layer.name:
        print("Modifying BatchNorm layer")
        # layer.beta_regularizer = l2(0.01)  # Should not be necessary
        layer.beta_constraint = max_norm(0.3)
    # if 'activ' in layer.name:
    #     # Try to see if we can make normalization unnecessary with this:
    #     print("Modifying Activation layer")
    #     layer.activity_regularizer = l2(0.01)

In [34]:
model.compile(SGD(lr=0.0001, momentum=0.9), 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(train_dataflow)

In [ ]:
model.save(os.path.join(save_path, 'inception_refined.h5'))